# Retrieval augmented generation

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'sample_rag'
ID_FIELD = '<var:id_field>' if not APPLY else 'id'

In [2]:
from superduper import superduper, CFG

CFG.bytes_encoding = 'str'
CFG.json_native = False

db = superduper('mongomock://test_db')

2025-Jan-13 12:30:50.28| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2025-Jan-13 12:30:50.35| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:64   | Building Data Layer
2025-Jan-13 12:30:50.35| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:79   | Data Layer built
2025-Jan-13 12:30:50.35| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:99   | Cluster initialized in 0.00 seconds.
2025-Jan-13 12:30:50.35| INFO     | Duncans-MBP.fritz.box| superduper.base.build:184  | Configuration: 
 +---------------+---------------------+
| Configuration |        Value        |
+---------------+---------------------+
|  Data Backend | mongomock://test_db |
+---------------+---------------------+


In [3]:
import json
import requests
import io

def getter():
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [4]:
if APPLY:
    data = getter()

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [5]:
if APPLY:
    from superduper import Document
    ids = db.execute(db[COLLECTION_NAME].insert([Document(r) for r in data]))

2025-Jan-13 12:30:51.37| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (table, sample_rag) not found in cache, loading from db
2025-Jan-13 12:30:51.37| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('table', 'sample_rag')) from metadata...
2025-Jan-13 12:30:51.37| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:331  | Table sample_rag does not exist, auto creating...
2025-Jan-13 12:30:51.89| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.torch` is deprecated and will be removed in a future release. Please insteall `superduper_torch` and use `from superduper_torch import *` instead.
2025-Jan-13 12:30:51.90| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.pillow` is deprecated and will be removed in a future release. Please insteall `superduper_pillow` and use `from superduper_pillow import *` instead.
2025-Jan-13 12:30:51.90| INFO     | Duncans

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [6]:
from superduper import Model


class Chunker(Model):
    chunk_size: int = 200
    signature: str = 'singleton'

    def predict(self, text):
        text = text.split()
        chunks = [' '.join(text[i:i + self.chunk_size]) for i in range(0, len(text), self.chunk_size)]
        return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [7]:
from superduper import Listener


upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db[COLLECTION_NAME].select(ID_FIELD, 'x'),
    key='x',
    identifier='chunker',
    flatten=True,
)

In [8]:
if APPLY:
    db.apply(upstream_listener, force=True)

2025-Jan-13 12:30:51.95| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, chunker) not found in cache, loading from db
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'chunker')) from metadata...
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new model:chunker:db30a0cd385740f3
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (listener, chunker) not found in cache, loading from db
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('listener', 'chunker')) from metadata...
2025-Jan-13 12:30:51.96| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (schema, _schema/_outputs__chunker__c287

187it [00:00, 47101.54it/s]

2025-Jan-13 12:30:51.98| INFO     | Duncans-MBP.fritz.box| superduper.components.model:672  | Adding 187 model outputs to `db`
2025-Jan-13 12:30:52.06| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:310  | Inserted 336 documents into _outputs__chunker__c287b7208d19494e
2025-Jan-13 12:30:52.06| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.queue:120  | Consumed all events


## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [9]:
import os

from superduper_openai import OpenAIEmbedding
from superduper.components.datatype import Vector

openai_embedding = OpenAIEmbedding(identifier='text-embedding-ada-002' , datatype=Vector(shape=(1536,)))

Sentence-transformers

In [10]:
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

sentence_transformers_embedding = SentenceTransformer(
    identifier="sentence-transformers-embedding",
    model="BAAI/bge-small-en",
    datatype=Vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

In [11]:
from superduper.components.model import ModelRouter
from superduper.components.datatype import Vector

embedding_model = ModelRouter(
    'embedding',
    models={'openai': openai_embedding, 'sentence_transformers': sentence_transformers_embedding},
    model='<var:embedding_model>' if not APPLY else 'openai',
    example='this is a test',
)

## Create vector-index

In [12]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs].select(ID_FIELD, '_source', upstream_listener.outputs),
        model=embedding_model,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

In [13]:
if APPLY:
    db.apply(vector_index, force=True)

2025-Jan-13 12:30:55.46| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:30:55.46| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:30:57.04| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:30:57.05| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:30:57.05| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:265  | Found identical model:chunker:db30a0cd385740f3
2025-Jan-13 12:30:57.06| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:341  | Found update model:chunker:db30a0cd385740f3
2025-Jan-13 12:30:57.06| WARNING  | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:87   | File /tmp/test_db/3d17ec69b27fea98f37d8b1e17e19691aaa0789b already exists
2025-Jan-13 12:30:57.06| WARNING  | Duncans-MBP.fr

vectorindex
└── chunker
    ├── status: update
    ├── changes
    │   └── _object: &:blob:3d17ec69b27fea98f37d8b1e17e19691aaa0789b
    └── type_id: model

2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:79   | Found these changes and/ or additions that need to be made:
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | METADATA EVENTS:
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:83   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [0]: model:chunker:db30a0cd385740f3: update
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   | [1]: datatype:vector[1536]:68a778b97abd494e: create ~ [6]
2025-Jan-13 12:31:00.32| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   | [2]

336it [00:00, 18935.91it/s]


2025-Jan-13 12:31:02.85| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1345 | Predicting with model openai


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.63s/it]


2025-Jan-13 12:31:09.37| INFO     | Duncans-MBP.fritz.box| superduper.components.model:672  | Adding 336 model outputs to `db`
2025-Jan-13 12:31:10.99| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:310  | Inserted 336 documents into _outputs__embeddinglistener__772ac0c8786e4153
2025-Jan-13 12:31:11.15| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.queue:120  | Consumed all events


<!-- TABS -->
## Build LLM

In [14]:
from superduper_openai import OpenAIChatCompletion

llm_openai = OpenAIChatCompletion(identifier='llm-openai', model='gpt-3.5-turbo')

In [15]:
from superduper_anthropic import AnthropicCompletions

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm_anthropic = AnthropicCompletions(identifier='llm-anthropic', model='claude-2.1', predict_kwargs=predict_kwargs)

In [16]:
from superduper_vllm import VllmCompletion

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm_vllm = VllmCompletion(
    identifier="llm-vllm",
    vllm_params={
        'model': 'TheBloke/Mistral-7B-Instruct-v0.2-AWQ',
        "gpu_memory_utilization": 0.7,
        "max_model_len": 1024,
        "quantization": "awq",
    },
    predict_kwargs=predict_kwargs,
)

2025-Jan-13 12:31:11.20| INFO     | Duncans-MBP.fritz.box| superduper_vllm.model:31   | Setting num_gpus to 1


In [17]:
llm = ModelRouter(
    'llm',
    models={
        'openai': llm_openai,
        'anthropic': llm_anthropic,
        'vllm': llm_vllm,
    },
    model='<var:llm_model>' if not APPLY else 'openai',
)

## Answer question with LLM

In [18]:
from superduper import model
from superduper.components.model import RAGModel

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'rag-model',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm,
)

In [19]:
if APPLY:
    db.apply(rag, force=True)

2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, llm-openai) not found in cache, loading from db
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'llm-openai')) from metadata...
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new model:llm-openai:6afc7764b1bd4861
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, llm-anthropic) not found in cache, loading from db
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'llm-anthropic')) from metadata...
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new model:llm-anthropic:68ef079ea5cd44d0
2025-Jan-13 12:31:11.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, llm-vllm) not found i

In [20]:
if APPLY:
    print(rag.predict('Tell me about vector-search'))

2025-Jan-13 12:31:11.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:802  | Getting vector-index
2025-Jan-13 12:31:11.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:810  | {}
2025-Jan-13 12:31:11.24| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1335 | Predicting with model openai
2025-Jan-13 12:31:11.55| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:802  | Getting vector-index
2025-Jan-13 12:31:11.56| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:810  | {}
2025-Jan-13 12:31:11.56| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1335 | Predicting with model openai
2025-Jan-13 12:31:11.95| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1335 | Predicting with model openai
Vector-search is a feature in Superduper that allows users to implement vector-search in their database using in-database functionality or through a sidecar implementation with `lance` and `FastAPI`. It involves pre

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [21]:
from superduper import Application

app = Application(
    'rag-app',
    components=[
        upstream_listener,
        vector_index,
        rag,
    ]
)

2025-Jan-13 12:31:13.84| INFO     | Duncans-MBP.fritz.box| superduper.components.application:39   | Resorting components based on topological order.
2025-Jan-13 12:31:13.84| INFO     | Duncans-MBP.fritz.box| superduper.components.application:56   | New order of components: ['listener:chunker:c287b7208d19494e', 'vector_index:vectorindex:dbeadf90742041b2', 'model:rag-model:079efb7994944436']


In [22]:
if APPLY:
    db.apply(app, force=True)

2025-Jan-13 12:31:13.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:31:13.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:31:13.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:31:13.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:31:15.46| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:31:15.46| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:31:15.47| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:31:15.47| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:31:17.05| WARNING  | Duncans-MBP.fritz.box| superduper.components.lis

rag-app
├── chunker
│   ├── status: update
│   ├── changes
│   │   └── _object: &:blob:3d17ec69b27fea98f37d8b1e17e19691aaa0789b
│   └── type_id: model
├── vector[1536]
│   ├── status: update
│   ├── changes
│   │   └── shape: (1536,)
│   └── type_id: datatype
├── vector[1024]
│   ├── status: update
│   ├── changes
│   │   └── shape: (1024,)
│   └── type_id: datatype
├── sentence-transformers-embedding
│   ├── status: update
│   ├── changes
│   │   ├── preferred_devices: ('cuda', 'mps', 'cpu')
│   │   └── postprocess: &:blob:ea6966092543717b01973a0109bd917ecd83bf38
│   └── type_id: model
├── embedding
│   ├── status: update
│   ├── changes
│   │   └── signature: *args,**kwargs
│   └── type_id: model
└── llm
    ├── status: update
    ├── changes
    │   └── signature: singleton
    └── type_id: model

2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:79   | Found these changes and/ or additions that need to be made:
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | METADATA EVENTS:
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:83   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [0]: model:chunker:db30a0cd385740f3: update
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [1]: datatype:vector[1536]:68a778b97abd494e: update
2025-Jan-13 12:31:21.91| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   | [2]: data

You can now load the model elsewhere and make predictions using the following command.

## Create template

In [24]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData


template = Template(
    'rag',
    template=app,
    default_tables=[Table(
        'sample_rag',
        schema=Schema(
            'sample_rag/schema',
            fields={'txt': 'str'},
        ),
        data=RemoteData(
            'superduper-docs',
            getter=getter,
        )
    )],
    substitutions={COLLECTION_NAME: 'table_name'},
    template_variables=['llm_model', 'embedding_model', 'table_name', 'id_field'],
    types={
        'id_field': {
            'type': 'str',
            'default': '_id',
        },
        'llm_model': {
            'type': 'str',
            'choices': ['openai', 'anthropic', 'vllm', 'llamacpp'],
            'default': 'openai',
        },
        'embedding_model': {
            'type': 'str',
            'choices': ['openai', 'sentence_transformers'],
            'default': 'openai',
        },
        'table_name': {
            'type': 'str',
            'default': 'sample_rag'
        },
    },
    db=db
)

2025-Jan-13 12:33:03.81| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:33:03.83| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:33:03.83| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf listener:chunker already exists
2025-Jan-13 12:33:03.83| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:33:03.83| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf model:chunker already exists
2025-Jan-13 12:33:03.84| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:33:03.84| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf sample-rag-select-id-x already exists
2025-Jan-13 12:33:03.84| WARNING  | Duncans-MBP.fritz.box| superduper.base.docume

In [25]:
template.export('.')

2025-Jan-13 12:35:53.15| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
